## Importing packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
# import matplotlib
import matplotlib.pyplot as plt
import pymc3 as pm
# import theano
import theano.tensor as tt

## Demand Estimation

### 1. Calculating demand elasticity and variables(a, b) with Baysian Method (MCMC)

The provided code block defines a function estimate_demand_parameters that uses Bayesian statistical modeling with PyMC3 to estimate demand elasticity parameters from a given pandas DataFrame. Specifically, it sets up a probabilistic model to understand how the demand for a product or service is influenced by its price, incorporating factors like price elasticity (eta), and constants a and b that define the demand curve. It employs priors, a demand function based on the exponential decay of price, and observes this against actual price data. After sampling from the posterior distribution to infer these parameters, it adds the estimated values as new columns (estimated_eta, estimated_a, estimated_b) to the original DataFrame. This approach provides a statistical framework to understand and quantify the relationship between price and demand in the given dataset.

In the code block, the demand function is defined as demand = a * pm.math.exp(-tt.abs_(eta) * tt.log(price_data)) + b. In this function, a, b, and eta are parameters that play distinct roles in modeling the relationship between price and demand:

- eta (Price Elasticity of Demand): This parameter represents the price elasticity of demand, a key concept in economics that describes how the quantity demanded of a good or service is affected by its price. In the model, eta is used within an exponential function to capture the sensitivity of demand to changes in price. A higher absolute value of eta indicates greater sensitivity.
- a (Demand Scaling Factor): The parameter a acts as a scaling factor for the demand. It adjusts the overall level of demand independently of the price. This factor can be influenced by variables other than price, such as product quality, consumer preferences, or market conditions.
- b (Baseline Demand): The parameter b represents a baseline or intercept term in the demand function. It signifies the level of demand when the effect of price is not considered (e.g., when price impact is zero or negligible). Essentially, it's a constant term that indicates the demand level due to factors other than price.  
  
Together, these parameters (eta, a, and b) in the demand function provide a comprehensive view of how demand varies with price, accounting for both the direct impact of price changes (through eta) and other factors influencing demand levels (a and b). By estimating these parameters using Bayesian methods in PyMC3, the function aims to create a nuanced and probabilistic understanding of demand dynamics in the given dataset.

In [10]:
data = pd.read_csv("../Cleaning_Engineering/base_cleaned.csv")
data.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
data = data.set_index('time_stamp_x')
data = data.sort_index()
data.head()

,distance,cab_type,source,destination,name,car_type,weekday,rush_hour,temp,clouds,pressure,rain,humidity,wind,is_raining,temp_groups,surge_multiplier,price
time_stamp_x,,,,,,,,,,,,,,,,,,
1543203646318,3.03,Lyft,Boston University,Theatre District,Lux Black XL,Luxury SUV,1,0,41.07,0.86,1014.39,NaN,0.92,1.36,0,40,1.0,34.0
1543203646319,1.30,Uber,South Station,Theatre District,Black,Luxury,1,0,40.86,0.87,1014.39,NaN,0.93,1.60,0,40,1.0,18.5
1543203646320,2.43,Lyft,Northeastern University,Beacon Hill,Lyft,Base,1,0,40.81,0.89,1014.35,NaN,0.93,1.36,0,40,1.0,10.5
1543203646320,2.71,Uber,Theatre District,Fenway,UberXL,Base XL,1,0,40.80,0.87,1014.39,NaN,0.93,1.55,0,40,1.0,32.0
1543203646320,2.71,Uber,Theatre District,Fenway,UberX,Base,1,0,40.80,0.87,1014.39,NaN,0.93,1.55,0,40,1.0,19.5


## Demand Parameters Estimation Experiment

### Overview

The `estimate_demand_parameters` function is designed to estimate demand parameters (eta, a, b) in a given dataset. This function uses Bayesian statistical modeling, specifically with the PyMC3 library, to infer these parameters based on observed price data and other relevant variables.

### Function Description

- **estimate_demand_parameters(dataframe, price_col)**:
  - **Purpose**: Estimates demand parameters eta, a, and b.
  - **Parameters**:
    - `dataframe`: A Pandas DataFrame containing the dataset.
    - `price_col`: The name of the column in the DataFrame that contains price data.

### Process

1. **Define Priors**:
   - The function begins by defining priors for the parameters using a Gamma distribution for eta and Uniform distributions for a and b.

2. **Model Specification**:
   - The price data is converted into a Theano tensor, which is a requirement for PyMC3 models.
   - The demand function is defined as `a * exp(-abs(eta) * log(price_data)) + b`.

3. **Observation and Sampling**:
   - Gaussian noise is assumed in the observed data.
   - The function then samples from the posterior distribution to estimate the parameters.

4. **Posterior Analysis**:
   - After sampling, the mean values of the posterior distributions for eta, a, and b are extracted.

5. **Dataframe Update**:
   - The function adds new columns to the original dataframe with estimated values for eta, a, and b.

### Output

- The function returns the updated DataFrame with the estimated parameters and the trace from the PyMC3 model.
- This methodology allows for a probabilistic understanding of demand as a function of price and can be particularly useful in dynamic pricing models.

### Dependencies

- PyMC3: A Python library for probabilistic programming.
- Theano: A Python library that allows you to define, optimize, and evaluate mathematical expressions, especially ones with multi-dimensional arrays.

### Usage

To use this function, ensure that your dataset contains at least the price column and other relevant features that may affect demand. The function will then provide estimates for the demand parameters, enhancing your analysis and modeling capabilities in pricing strategies.


In [11]:

def estimate_demand_parameters(dataframe, price_col):
    """
    Estimates the demand parameters (eta, a, b) using PyMC3 and adds them as new columns in the dataframe.

    :param dataframe: Pandas DataFrame containing the data
    :param price_col: Name of the column containing price data
    :return: DataFrame with new columns for eta, a, b estimates
    """
    with pm.Model() as model:
        # Define priors
        eta_shape = 2 
        eta_rate = 5  
        eta = pm.Gamma('eta', alpha=eta_shape, beta=eta_rate) + 0.25
        a = pm.Uniform('a', lower=0, upper=100)
        b = pm.Uniform('b', lower=0, upper=20)

        # Convert the price data to a theano tensor
        price_data = tt.as_tensor_variable(dataframe[price_col].values)

        surge_multiplier_data = tt.as_tensor_variable(dataframe['surge_multiplier'].values)
        # Define the demand function using pm.math.exp for exponentiation
        demand = a * pm.math.exp(-tt.abs_(eta) * tt.log(price_data)) + b

        # Assuming Gaussian noise in the observed data
        observed = pm.Normal('observed', mu=demand, sd=1, observed=dataframe[price_col])

        # Sample from the posterior
        trace = pm.sample(1000, tune=1000, return_inferencedata=False)

    # Extracting the mean of the posterior distributions
    eta_mean = np.mean(trace['eta'])
    a_mean = np.mean(trace['a'])
    b_mean = np.mean(trace['b'])

    # Adding new columns to the dataframe
    dataframe['estimated_eta'] = eta_mean
    dataframe['estimated_a'] = a_mean
    dataframe['estimated_b'] = b_mean

    return dataframe, trace

### !!! Warning: It will take around an hour to finish to generate estimated eta, a and b
### !!! Please use the ready-made csv file to analyze (/Cleaning_Engineering/added_demand_estimates.csv)

In [13]:
# Generate a list of all unique combinations of car_type, source, destination
lst_car_type = list(set(data.car_type))
lst_source = list(set(data.source))
lst_destination = list(set(data.destination))

results_list = []

# Iterate over all combinations
for car_type in lst_car_type:
    for source in lst_source:
        for destination in lst_destination:
            # Filter the data for the current combination
            filtered_data = data[(data['car_type'] == car_type) & (data['source'] == source) & (data['destination'] == destination)]

            # Treat the combination as valid only if data points exist
            if not filtered_data.empty:
                try:
                    result_df, _ = estimate_demand_parameters(filtered_data, 'price')
                    results_list.append(result_df)
                except Exception as e:
                    print(f"Error processing combination: CarType={car_type}, Source={source}, Destination={destination}")
                    print(f"Error message: {e}")

# Combine the results into a single DataFrame
combined_df = pd.concat(results_list, ignore_index=True)

# Calculate the estimated demand for each combination
combined_df['estimated_demand'] = combined_df['estimated_a'] * np.exp(-np.abs(combined_df['estimated_eta']) * np.log(combined_df['price'])) + combined_df['estimated_b']

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a, eta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 51 seconds.
/Users/jacob/opt/anaconda3/envs/SIADS/lib/python3.8/site-packages/arviz/utils.py:175: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 18 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/3

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 45 seconds.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 46 seconds.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There were 11 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 45 seconds.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

...

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 50 seconds.
There were 83 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6926604933636774, but should be close to 0.8. Try to increase the number of tuning steps.
There were 58 divergences after tuning. Increase `target_accept` or reparameterize.
There were 40 divergences after tuning. Increase `target_accept` or reparameterize.
There were 24 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 49 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_a'] = a_mean

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 50 seconds.
There were 43 divergences after tuning. Increase `target_accept` or reparameterize.
There were 54 divergences after tuning. Increase `target_accept` or reparameterize.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
There were 31 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 51 seconds.
There were 33 divergences after tuning. Increase `target_accept` or reparameterize.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
There were 38 divergences after tuning. Increase `target_accept` or reparameterize.
There were 47 divergences after tuning. Increase `target_accept` or reparameterize.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['estimated_eta'] = eta_mean
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 49 seconds.
There were 50 divergences after tuning. Increase `target_accept` or reparameterize.
There were 174 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6446406380094439, but should be close to 0.8. Try to increase the number of tuning steps.
There were 49 divergences after tuning. Increase `target_accept` or reparameterize.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.
/var/folders/ym/g7y_9d4j7q9d413s_kdqly_m0000gn/T/ipykernel_59617/338695646.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [15]:
combined_df.shape

(637976, 22)

In [16]:
combined_df.head()

,distance,cab_type,source,destination,name,car_type,weekday,rush_hour,temp,clouds,...,humidity,wind,is_raining,temp_groups,surge_multiplier,price,estimated_eta,estimated_a,estimated_b,estimated_demand
0,3.01,Uber,North Station,Boston University,Black SUV,Luxury SUV,1,0,40.51,1.00,...,0.92,1.35,0,40,1.0,35.0,0.000301,34.518257,19.99128,54.472641
1,3.01,Uber,North Station,Boston University,Black SUV,Luxury SUV,1,0,40.51,1.00,...,0.92,1.35,0,40,1.0,32.5,0.000301,34.518257,19.99128,54.473410
2,3.01,Uber,North Station,Boston University,Black SUV,Luxury SUV,1,0,40.54,1.00,...,0.92,1.42,0,40,1.0,35.0,0.000301,34.518257,19.99128,54.472641
3,3.15,Lyft,North Station,Boston University,Lux Black XL,Luxury SUV,1,1,41.10,0.97,...,0.92,1.82,0,40,1.0,34.0,0.000301,34.518257,19.99128,54.472942
4,3.09,Lyft,North Station,Boston University,Lux Black XL,Luxury SUV,1,1,41.10,0.97,...,0.92,1.82,0,40,1.0,32.5,0.000301,34.518257,19.99128,54.473410


In [ ]:
# Add a new column for the estimated demand
combined_df['estimated_demand'] = combined_df['estimated_a'] * np.exp(-np.abs(combined_df['estimated_eta']) * np.log(combined_df['price'])) + combined_df['estimated_b']
combined_df.head()

In [17]:
# Export to CSV
combined_df.to_csv("../Cleaning_Engineering/added_demand_estimates_eta_variation_ver2.csv", index=False)

In [18]:
combined_df.shape

(637976, 22)

In [20]:
combined_df.isna().sum()

distance                 0
cab_type                 0
source                   0
destination              0
name                     0
car_type                 0
weekday                  0
rush_hour                0
temp                     0
clouds                   0
pressure                 0
rain                544429
humidity                 0
wind                     0
is_raining               0
temp_groups              0
surge_multiplier         0
price                    0
estimated_eta            0
estimated_a              0
estimated_b              0
estimated_demand         0
dtype: int64

Note that the file is large, so we installed git LFS and committed the file.
The CSV file was about 100 megabytes, but the generated list was almost 650 megabytes, so when I made it into a PKL file, I split it into 90 megabytes and saved it.

In [41]:
import pickle
import os
import math

def load_large_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

def save_split_pickle(data, base_filename, max_size_mb=90):
    # Calculate the number of parts to split the data into
    total_size_bytes = len(pickle.dumps(data))
    total_size_mb = total_size_bytes / (1024 * 1024)
    num_parts = math.ceil(total_size_mb / max_size_mb)

    # Store the data in multiple files
    part_length = len(data) // num_parts
    for i in range(num_parts):
        part_data = data[i * part_length: (i + 1) * part_length]
        with open(f'{base_filename}_part{i}.pkl', 'wb') as file:
            pickle.dump(part_data, file)

    # Store the remaining data
    if len(data) % num_parts != 0:
        part_data = data[num_parts * part_length:]
        with open(f'{base_filename}_part0{num_parts}.pkl', 'wb') as file:
            pickle.dump(part_data, file)


save_split_pickle(results_list, "../Cleaning_Engineering/result_list", max_size_mb=90)


It is highly recommended to use generated csv file(added_demand_estimates.csv) instead of running the above codes again.